# Set connection with GitHub

In [1]:
# ! cd /kaggle/working && rm -rf RECsys_Challenge2024

In [2]:
from kaggle_secrets import UserSecretsClient

token = UserSecretsClient().get_secret("Token")

! git clone https://{token}@github.com/madratak/RECsys_Challenge2024.git

Cloning into 'RECsys_Challenge2024'...
remote: Enumerating objects: 6196, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 6196 (delta 35), reused 16 (delta 6), pack-reused 6089 (from 4)
Receiving objects: 100% (6196/6196), 399.83 MiB | 37.79 MiB/s, done.
Resolving deltas: 100% (3415/3415), done.
Updating files: 100% (486/486), done.


In [3]:
! pip install PyGithub requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 23.4 MB/s eta 0:00:00


In [4]:
%cd /kaggle/working/RECsys_Challenge2024 
! python run_compile_all_cython.py

/kaggle/working/RECsys_Challenge2024
run_compile_all_cython: Found 11 Cython files in 5 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/usr/bin/python3'
Compiling [1/11]: MatrixFactorizationImpressions_Cython_Epoch.pyx... 
In file included from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorizationImpressions_Cython_Epoch.c:1252:
/usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wcpp-Wcpp]8;;]
   17 | #warning "Using deprec

In [5]:
from Utils.notebookFunctions import *
import numpy as np
import pandas as pd
import scipy.sparse as sps
import time
import shutil
import optuna
import json
import os
from Utils.seconds_to_biggest_unit import seconds_to_biggest_unit


K_PATH = '/kaggle/working/RECsys_Challenge2024'
GH_PATH = 'TrainedModels/WithKFCV/SLIM'

np.random.seed(42)

## Import the repository **RECsys_Challenge2024**

In [6]:
repo = get_repo_from_github(token)

Repository 'RECsys_Challenge2024' found.


In [7]:
config = {
    'model': 'SLIM_BPR',
    'n_folds': 5,
    'metric': 'MAP',
    'tune_parameters': True,
    'database_path': '/kaggle/working/history_SLIM_BPR_MAP.db',
    'copy_prev_best_params': False,
    'tune_best_params': True,
    'save_github': True
}

Import the database where previous tuning trials have been saved.

In [8]:
try:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/history_{config["model"]}_{config["metric"]}.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

# Construction of URM and ICM matrices

In [9]:
URM_all_dataframe = pd.read_csv("/kaggle/working/RECsys_Challenge2024/Dataset/data_train.csv")

n_users = len(URM_all_dataframe["user_id"].unique())
n_items = len(URM_all_dataframe["item_id"].unique())

URM_all = sps.csr_matrix((URM_all_dataframe["data"].values, 
                          (URM_all_dataframe["user_id"].values, URM_all_dataframe["item_id"].values)),
                        shape = (n_users, n_items))

URM_all

<35736x38121 sparse matrix of type '<class 'numpy.float64'>'
	with 1764607 stored elements in Compressed Sparse Row format>

In [10]:
ICM_dataframe = pd.read_csv("/kaggle/working/RECsys_Challenge2024/Dataset/data_ICM_metadata.csv")

n_items = len(ICM_dataframe["item_id"].unique())
n_features = len(ICM_dataframe["feature_id"].unique())

ICM_all = sps.csr_matrix((ICM_dataframe["data"].values, 
                          (ICM_dataframe["item_id"].values, ICM_dataframe["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all

<38121x94331 sparse matrix of type '<class 'numpy.float64'>'
	with 2940040 stored elements in Compressed Sparse Row format>

# Training

In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Data_manager.split_functions.split_train_k_folds import split_train_k_folds

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

folds = split_train_k_folds(URM_all, k=config['n_folds'])

In [12]:
from Recommenders.SLIM.SLIM_BPR_Python import SLIM_BPR_Python

def objective_function_SLIM_BPR(optuna_trial):

    full_hyperp = {
                    "topK": optuna_trial.suggest_int("topK", 0, 500),
                    "epochs": optuna_trial.suggest_int("epochs", 1, 50),
                    "lambda_i": optuna_trial.suggest_float("lambda_i", 1e-5, 1e-1, log=True),
                    "lambda_j": optuna_trial.suggest_float("lambda_j", 1e-5, 1e-1, log=True),
                    "learning_rate": optuna_trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
                }
    
    validation_results = []
    
    for fold_idx, (URM_train_fold, URM_validation_fold) in enumerate(folds):
    
        recommender_instance = SLIM_BPR_Python(URM_train_fold)
        recommender_instance.fit(**full_hyperp)
        
        evaluator = EvaluatorHoldout(URM_validation_fold, cutoff_list=[10])
        result_df, _ = evaluator.evaluateRecommender(recommender_instance)
        
        validation_results.append(result_df.loc[10]["MAP"])
    
    return np.mean(validation_results)

In [13]:
if config['tune_parameters']:
    
    optuna_study = optuna.create_study(direction='maximize', study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)

    optuna_study.optimize(objective_function_SLIM_BPR, n_trials=15)

[I 2025-01-05 12:18:04,227] A new study created in RDB with name: hyperparameters_tuning_SLIM_BPR_MAP


Epoch 1, Iteration 35736 in 10.55 seconds. Samples per second 3386.50
Epoch 2, Iteration 35736 in 7.62 seconds. Samples per second 4688.94
Epoch 3, Iteration 35736 in 6.25 seconds. Samples per second 5721.77
Epoch 4, Iteration 35736 in 5.17 seconds. Samples per second 6912.88
Epoch 5, Iteration 35736 in 4.56 seconds. Samples per second 7837.53
Epoch 6, Iteration 35736 in 4.15 seconds. Samples per second 8614.82
Epoch 7, Iteration 35736 in 3.86 seconds. Samples per second 9246.18
Epoch 8, Iteration 35736 in 3.63 seconds. Samples per second 9841.89
Epoch 9, Iteration 35736 in 3.47 seconds. Samples per second 10300.99
Epoch 10, Iteration 35736 in 3.46 seconds. Samples per second 10320.17
Epoch 11, Iteration 35736 in 3.29 seconds. Samples per second 10845.85
Epoch 12, Iteration 35736 in 3.23 seconds. Samples per second 11052.65
Epoch 13, Iteration 35736 in 3.20 seconds. Samples per second 11152.37
Epoch 14, Iteration 35736 in 3.17 seconds. Samples per second 11273.02
Epoch 15, Iteration 35

[I 2025-01-05 12:45:37,335] Trial 0 finished with value: 0.030840353646130032 and parameters: {'topK': 362, 'epochs': 23, 'lambda_i': 0.052867545767967615, 'lambda_j': 0.010496161067592736, 'learning_rate': 0.06949688514175209}. Best is trial 0 with value: 0.030840353646130032.


Epoch 1, Iteration 35736 in 10.11 seconds. Samples per second 3536.23
Epoch 2, Iteration 35736 in 7.44 seconds. Samples per second 4800.03
Epoch 3, Iteration 35736 in 5.80 seconds. Samples per second 6157.84
Epoch 4, Iteration 35736 in 5.00 seconds. Samples per second 7140.23
Epoch 5, Iteration 35736 in 4.39 seconds. Samples per second 8145.88
Epoch 6, Iteration 35736 in 4.03 seconds. Samples per second 8865.29
Epoch 7, Iteration 35736 in 3.82 seconds. Samples per second 9366.97
Epoch 8, Iteration 35736 in 3.60 seconds. Samples per second 9937.49
Epoch 9, Iteration 35736 in 3.55 seconds. Samples per second 10063.64
Epoch 10, Iteration 35736 in 3.48 seconds. Samples per second 10261.03
Epoch 11, Iteration 35736 in 3.32 seconds. Samples per second 10767.31
Epoch 12, Iteration 35736 in 3.30 seconds. Samples per second 10815.50
Epoch 13, Iteration 35736 in 3.24 seconds. Samples per second 11039.22
Epoch 14, Iteration 35736 in 3.20 seconds. Samples per second 11168.65
Epoch 15, Iteration 35

[I 2025-01-05 13:17:09,808] Trial 1 finished with value: 0.03301220945830492 and parameters: {'topK': 48, 'epochs': 48, 'lambda_i': 4.501902294751744e-05, 'lambda_j': 0.033796914839612925, 'learning_rate': 0.0031929654934849884}. Best is trial 1 with value: 0.03301220945830492.


...

[I 2025-01-05 18:19:26,800] Trial 12 finished with value: 0.030198658109675335 and parameters: {'topK': 89, 'epochs': 16, 'lambda_i': 1.164812826726499e-05, 'lambda_j': 0.01108009813759228, 'learning_rate': 0.0072048463400766795}. Best is trial 1 with value: 0.03301220945830492.


Epoch 1, Iteration 35736 in 10.03 seconds. Samples per second 3562.97
Epoch 2, Iteration 35736 in 7.28 seconds. Samples per second 4908.09
Epoch 3, Iteration 35736 in 5.83 seconds. Samples per second 6125.86
Epoch 4, Iteration 35736 in 4.98 seconds. Samples per second 7175.79
Epoch 5, Iteration 35736 in 4.37 seconds. Samples per second 8168.40
Epoch 6, Iteration 35736 in 4.05 seconds. Samples per second 8834.04
Epoch 7, Iteration 35736 in 3.80 seconds. Samples per second 9402.96
Epoch 8, Iteration 35736 in 3.57 seconds. Samples per second 10014.20
Epoch 9, Iteration 35736 in 3.53 seconds. Samples per second 10131.67
Epoch 10, Iteration 35736 in 3.40 seconds. Samples per second 10495.43
Epoch 11, Iteration 35736 in 3.32 seconds. Samples per second 10754.11
Epoch 12, Iteration 35736 in 3.29 seconds. Samples per second 10854.82
Epoch 13, Iteration 35736 in 3.30 seconds. Samples per second 10817.34
Epoch 14, Iteration 35736 in 3.24 seconds. Samples per second 11025.57
Epoch 15, Iteration 3

[I 2025-01-05 18:52:24,501] Trial 13 finished with value: 0.031999829758385716 and parameters: {'topK': 122, 'epochs': 36, 'lambda_i': 6.115559892064658e-05, 'lambda_j': 0.016618521383949855, 'learning_rate': 0.009202181787699529}. Best is trial 1 with value: 0.03301220945830492.


Epoch 1, Iteration 35736 in 10.03 seconds. Samples per second 3562.51
Epoch 2, Iteration 35736 in 7.25 seconds. Samples per second 4927.38
Epoch 3, Iteration 35736 in 5.87 seconds. Samples per second 6085.83
Epoch 4, Iteration 35736 in 4.96 seconds. Samples per second 7202.81
Epoch 5, Iteration 35736 in 4.44 seconds. Samples per second 8056.01
Epoch 6, Iteration 35736 in 4.03 seconds. Samples per second 8869.35
Epoch 7, Iteration 35736 in 3.91 seconds. Samples per second 9145.24
Epoch 8, Iteration 35736 in 9.26 seconds. Samples per second 3861.14
Epoch 9, Iteration 35736 in 3.94 seconds. Samples per second 9080.44
Epoch 10, Iteration 35736 in 3.65 seconds. Samples per second 9800.88
Epoch 11, Iteration 35736 in 3.51 seconds. Samples per second 10190.26
Epoch 12, Iteration 35736 in 3.37 seconds. Samples per second 10588.58
Epoch 13, Iteration 35736 in 3.32 seconds. Samples per second 10776.76
Epoch 14, Iteration 35736 in 3.38 seconds. Samples per second 10573.10
Epoch 15, Iteration 3573

[I 2025-01-05 19:24:11,450] Trial 14 finished with value: 0.030917047411810217 and parameters: {'topK': 127, 'epochs': 35, 'lambda_i': 0.00013558305148759082, 'lambda_j': 0.0330457027192364, 'learning_rate': 0.002958943615098495}. Best is trial 1 with value: 0.03301220945830492.


## Some optuna visualizations on recommender parameters

In [14]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_slice(optuna_study)
fig.show()

In [15]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_param_importances(optuna_study)
fig.show()

## Let's train the recommender with best parameter values

In [16]:
if config['tune_best_params']:

    if config['tune_parameters']:
        best_params = optuna_study.best_trial.params
    else: 
        with open(f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/best_params_{config["model"]}_{config["metric"]}.json', 'r') as best_params_json:
            best_params = json.load(best_params_json)

    recommender_instance = SLIM_BPR_Python(URM_train + URM_validation)
    recommender_instance.fit(**best_params)

Epoch 1, Iteration 35736 in 10.82 seconds. Samples per second 3302.76
Epoch 2, Iteration 35736 in 7.59 seconds. Samples per second 4706.34
Epoch 3, Iteration 35736 in 6.04 seconds. Samples per second 5915.70
Epoch 4, Iteration 35736 in 5.12 seconds. Samples per second 6976.37
Epoch 5, Iteration 35736 in 4.62 seconds. Samples per second 7738.20
Epoch 6, Iteration 35736 in 4.08 seconds. Samples per second 8749.08
Epoch 7, Iteration 35736 in 3.86 seconds. Samples per second 9264.81
Epoch 8, Iteration 35736 in 3.64 seconds. Samples per second 9812.97
Epoch 9, Iteration 35736 in 3.55 seconds. Samples per second 10077.36
Epoch 10, Iteration 35736 in 3.44 seconds. Samples per second 10387.07
Epoch 11, Iteration 35736 in 3.35 seconds. Samples per second 10675.80
Epoch 12, Iteration 35736 in 3.35 seconds. Samples per second 10669.43
Epoch 13, Iteration 35736 in 8.04 seconds. Samples per second 4445.05
Epoch 14, Iteration 35736 in 5.87 seconds. Samples per second 6083.70
Epoch 15, Iteration 3573

# Testing

Create the recommendations for the submission. 

In [17]:
if config['tune_best_params']:

    data_target_users_test = pd.read_csv('/kaggle/working/RECsys_Challenge2024/Dataset/data_target_users_test.csv')
    create_submission(data_target_users_test, recommender_instance, f'/kaggle/working/submission_{config["model"]}_{config["metric"]}.csv')

Submission file saved as /kaggle/working/submission_SLIM_BPR_MAP.csv


# Save Version on GitHub 

Write or import a json file where best hyperparameters are saved. 

In [18]:
if config['tune_parameters']:
    with open(f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json', 'w') as params_file:
        json.dump(optuna_study.best_params, params_file)
        
    if config['save_github']:
        upload_file(
            f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json', 
            f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/best_params_{config["model"]}_{config["metric"]}.json', 
            f'{config["model"]}_{config["metric"]} tuning results (from kaggle notebook)',
            repo
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/'\
        f'best_params_{config["model"]}_{config["metric"]}.json', 
        f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json'
    )

File 'TrainedModels/WithKFCV/SLIM/SLIM_BPRRecommender/OptimizingMAP/best_params_SLIM_BPR_MAP.json' created successfully.


Save the history of the tuned model.

In [19]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/history_{config["model"]}_{config["metric"]}.db',
        f'Tuning {config["model"]}_{config["metric"]} db updated results (from kaggle notebook)',
        repo
    )

File 'TrainedModels/WithKFCV/SLIM/SLIM_BPRRecommender/OptimizingMAP/history_SLIM_BPR_MAP.db' created successfully.


Save the best trained model and its submission.

In [20]:
if config['save_github'] and config['tune_best_params']: 
    upload_file(
                f'/kaggle/working/submission_{config["model"]}_{config["metric"]}.csv', 
                f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/Submission/submission_{config["model"]}_{config["metric"]}.csv', 
                f'New {config["model"]}_{config["metric"]} submission (from kaggle notebook)',
                repo
            )

File 'TrainedModels/WithKFCV/SLIM/SLIM_BPRRecommender/OptimizingMAP/Submission/submission_SLIM_BPR_MAP.csv' created successfully.
